# **Human Pose Prediction**

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import csv

import numpy as np

import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from IPython.display import clear_output

import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# writer = SummaryWriter()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir -p data
!tar -xf /content/drive/Shared\ drives/Computer_Vision_Final_Project/data/train/dataset_10_skip_train_clean.pkl.tar.gz -C /content/data
with open('/content/data/dataset_10_skip_train_clean.pkl', 'rb') as f:
  train_dataloader = pickle.load(f)

!tar -xf /content/drive/Shared\ drives/Computer_Vision_Final_Project/data/test/dataset_10_skip_test_clean.pkl.tar.gz -C /content/data
with open('/content/data/dataset_10_skip_test_clean.pkl', 'rb') as f:
  test_dataloader = pickle.load(f)


In [0]:
print(len(train_dataloader))
print(len(test_dataloader))

20771
19793


# **Dataset Loader**

In [0]:
maxs_ = np.zeros(66)
train_dataset = []

for i in range(len(train_dataloader)):
    if len(train_dataloader[i]['Input']) == 10 and len(train_dataloader[i]['Output']) == 10:
        inp_ = np.array(train_dataloader[i]['Input'])[:,:66]
        inp_max = np.amax(inp_, axis=0)
        out_ = np.array(train_dataloader[i]['Output'])[:,:66]
        out_max = np.amax(out_, axis=0)
        data = {}
        data['Input'] = inp_
        data['Output'] = out_
        train_dataset.append(data)
        maxs_ = np.maximum(maxs_, np.abs(inp_max))
        maxs_ = np.maximum(maxs_, np.abs(out_max))

test_dataset = []
for i in range(len(test_dataloader)):
    if len(test_dataloader[i]['Input']) == 10 and len(test_dataloader[i]['Output']) == 10:
        inp_ = np.array(test_dataloader[i]['Input'])[:,:66]
        inp_max = np.amax(inp_, axis=0)
        out_ = np.array(test_dataloader[i]['Output'])[:,:66]
        out_max = np.amax(out_, axis=0)
        data = {}
        data['Input'] = inp_
        data['Output'] = out_
        test_dataset.append(data)
        maxs_ = np.maximum(maxs_, np.abs(inp_max))
        maxs_ = np.maximum(maxs_, np.abs(out_max))

# print(maxs_)
# print(len(train_dataset))
# print(len(test_dataset))
# print(train_dataset[10]['Input'].shape)
# print(train_dataset[15]['Output'].shape)

In [0]:
def load_data(batch_size, inp_seq_len, out_seq_len):
    data_in = []
    data_out = []
    used_alphas = []
    while(len(data_in) != batch_size or len(data_out) != batch_size):
        alpha = np.random.randint(0, len(train_dataset))
        if alpha in used_alphas:
            continue
        used_alphas.append(alpha)
        inp_ = train_dataset[alpha]['Input'][:inp_seq_len,:]/maxs_
        out_ = train_dataset[alpha]['Output'][:out_seq_len,:]/maxs_
        if(inp_.shape[0] != inp_seq_len or out_.shape[0] != out_seq_len):
            continue
        data_in.append(list(inp_))
        data_out.append(list(out_))

    data_in = torch.tensor(data_in, dtype=torch.float32)
    data_out = torch.tensor(data_out, dtype=torch.float32)
    
    data_in = data_in.permute(1,0,2)
    data_out = data_out.permute(1,0,2)

    return data_in, data_out

# class WTF_DS(Dataset):
#     """AMASS: a pytorch loader for unified human motion capture dataset. http://amass.is.tue.mpg.de/"""

#     def __init__(self, dataset_dir, inp_seq_len, out_seq_len, max_s):
#         with open(dataset_dir, 'rb') as f:
#             self.dataset = pickle.load(f)
#         self.inp_seq_len = inp_seq_len
#         self.out_seq_len = out_seq_len
#         self.max_s = max_s

#     def __len__(self):
#        return len(self.dataset)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#         sample = {}
#         sample['Input'] = torch.from_numpy(self.dataset[idx]['Input'][:self.inp_seq_len,:66])
#         sample['Output'] = torch.from_numpy(self.dataset[idx]['Output'][:self.out_seq_len,:66])

#         return sample

# tr_ds = WTF_DS('/content/data/dataset_10_skip_train_clean.pkl', 10, 5)

In [0]:
#tr_ds[0]
tr_loader = DataLoader(tr_ds, batch_size = 10, shuffle=True, num_workers=4)
for idx, batch in enumerate(tr_loader):
    print(batch['Input'].shape)
    break

torch.Size([10, 10, 66])


# **Architecture**

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, input_depth, batch_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.linear = nn.Linear(input_size, input_depth)
        self.gru = nn.GRU(input_depth, hidden_size, 2)

    def forward(self, input_, hidden):
        encoder_inputs = self.linear(input_)
        output, hidden = self.gru(encoder_inputs, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, self.batch_size, self.hidden_size, device=device)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, input_depth, batch_size, out_seq_len):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.out_seq_len = out_seq_len

        self.linear = nn.Linear(hidden_size, input_depth, bias=False)
        self.gru = nn.GRU(input_depth, hidden_size, 2)
        self.dec_out = nn.Linear(hidden_size, input_depth)
        self.out = nn.Linear(input_depth, output_size)

    def forward(self, input_, hidden):
        outputs = []
        decoder_input = self.linear(input_)
        for i in range(self.out_seq_len):
            output, hidden = self.gru(decoder_input, hidden)
            decoder_input = self.dec_out(output)
            
            output = self.out(decoder_input)
            outputs.append(output)
        return torch.stack(outputs).squeeze(), hidden

    def initHidden(self):
        return torch.zeros(2, self.batch_size, self.hidden_size, device=device)

In [0]:
class GeneratorRNN(nn.Module):
    
    def __init__(self, batch_size, hidden_size, input_depth, input_size, input_seq_length, output_seq_length):
        super(GeneratorRNN, self).__init__()
        self.encoder = EncoderRNN(input_size, hidden_size, input_depth, batch_size)
        self.decoder = DecoderRNN(hidden_size, input_size, input_depth, batch_size, output_seq_length)
        
        self.lin0 = nn.Linear(input_depth, hidden_size, bias=False)
        self.lin1 = nn.Linear(input_depth, hidden_size, bias=False)
        
        self.num_layers = 2
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.input_seq_length = input_seq_length
        self.output_seq_length = output_seq_length
        self.input_depth = input_depth
        self.input_size = input_size
    
    def forward(self, inputs_, hidden, z):

        encoder_output, encoder_state = self.encoder(inputs_, hidden)
        encoder_state[0] += self.lin0(z)
        encoder_state[1] += self.lin1(z)

        decoder_output, decoder_hidden = self.decoder(encoder_output[-1,:,:].unsqueeze(0), encoder_state)

        return decoder_output


In [0]:
class NNDiscriminator(nn.Module):
    
    def __init__(self, input_size, input_depth, seq_length, scope=""):
        super(NNDiscriminator, self).__init__()
        self.num_neurons = 512
        self.num_layers = 3
        self.seq_length = seq_length
        self.input_depth = input_depth
        self.scope = scope

        if scope == "":
            self.model = nn.Sequential(nn.Linear(input_size * seq_length, input_depth, bias=False),
                                       nn.Linear(input_depth, self.num_neurons),
                                       nn.ReLU(),
                                       nn.Linear(self.num_neurons, self.num_neurons),
                                       nn.ReLU(),
                                       nn.Linear(self.num_neurons, self.num_neurons),
                                       nn.ReLU(),
                                       nn.Linear(self.num_neurons, 1))
        elif scope == "prob":
            self.model = nn.Sequential(nn.Linear(input_size * seq_length, input_depth, bias=False),
                                       nn.Linear(input_depth, self.num_neurons),
                                       nn.ReLU(),
                                       nn.Linear(self.num_neurons, self.num_neurons),
                                       nn.ReLU(),
                                       nn.Linear(self.num_neurons, self.num_neurons),
                                       nn.ReLU(),
                                       nn.Linear(self.num_neurons, 1),
                                       nn.Sigmoid())

    def forward(self, input_):
        batch_size = input_.shape[1]
        input_ = torch.reshape(input_, (batch_size, input_.shape[0]*input_.shape[2]))
        return self.model(input_)
    
    def weights(self):
        w = []
        for i in range(8):
            try:
              w.append(self.model[i].weight.data)
            except:
              continue
        return w



# **Train**

In [0]:
def gradient_penalty(d_inputs, d_fake_inputs, critic):
    
    batch_size = d_inputs.shape[0]

    alpha = torch.rand(batch_size, 1, 1).cuda()
    
    alpha = alpha.expand(d_inputs.size())

    d_inputs_hat = alpha * d_inputs + (1 - alpha) * d_fake_inputs
    d_inputs_hat = torch.autograd.Variable(d_inputs_hat, requires_grad=True)

    d_outputs_hat = critic(d_inputs_hat)
    
    gradients = torch.autograd.grad(d_outputs_hat, d_inputs_hat, torch.ones(d_outputs_hat.size()).cuda(),
                                    create_graph=True, retain_graph=True, only_inputs=True)
    
    gp = ((gradients[0].norm(2, dim=1) - 1) ** 2).mean()
    
    return gp

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [0]:
def train(G, D, C, num_epochs, resume=False):
  # # Initialization
  hist_D_losses = []
  hist_G_losses = []
  hist_C_losses = []
  
  hist_gcons_losses = []
  hist_gl2_losses = []
  hist_ggan_losses = []

  hist_cw_losses = []
  hist_cgp_losses = []
  hist_cl2_losses = []

  hist_dgan_losses = []
  hist_dl2_losses = []

  minibatch_size = 20           # batch size change
  lr_init = 5e-5
  hidden_size = 1024            # number of hidden neurons in GRU
  output_sequence_length = 10    # See if needs to change
  input_sequence_length = 10

  d_lr = lr_init
  g_lr = lr_init    

  G_optimizer = torch.optim.Adam(G.parameters(), lr=g_lr)
  D_optimizer = torch.optim.Adam(D.parameters(), lr=d_lr)
  C_optimizer = torch.optim.Adam(C.parameters(), lr=d_lr/2.0)

  if resume == True:
    # Load model
    checkpoint = torch.load('/content/drive/Shared drives/Computer_Vision_Final_Project/saved_weights/c05g01l10_epoch20.pt')
    C.load_state_dict(checkpoint['C'])
    D.load_state_dict(checkpoint['D'])
    G.load_state_dict(checkpoint['G'])
    C_optimizer.load_state_dict(checkpoint['C_optimizer'])
    D_optimizer.load_state_dict(checkpoint['D_optimizer'])
    G_optimizer.load_state_dict(checkpoint['G_optimizer'])

    C.train()
    D.train()
    G.train()

  print('training start!')
  start_time = time.time()
  for epoch in range(num_epochs):
      print('Start training epoch %d' % (epoch + 1))
      D_losses = []
      G_losses = []
      C_losses = []
      epoch_start_time = time.time()
      num_iter = 0
      for num_iter in range(1000):        # train_loader = train data            ## Decide whether to use frames or differences of frames
          
          # if(num_iter % 100 == 0 and num_iter != 0):
          print("iteration", num_iter)
            # print("generator loss %2.2f"%g_loss)
            # print("critic loss %2.2f"%d_loss)
            # print("discriminator loss %2.2f"%d_loss_prob)

          g_in, real_op = load_data(minibatch_size, input_sequence_length, output_sequence_length)
              
          g_in, real = Variable(g_in.cuda()), Variable(real_op.cuda())   #converting into tensors
          input_sequence_length = g_in.shape[0]              ######CHECK THE DIM
            
          
          # train discriminator D
          #++++++++++++++++++++++++++++++++++++++++++++++#
          #++++++++++++++++++++++++++++++++++++++++++++++#
          # ============== YOUR CODE HERE ============== # 
          # set zero gradient
          G_optimizer.zero_grad()
          D_optimizer.zero_grad()
          C_optimizer.zero_grad()

          # Generate z and hidden
          hidden = torch.zeros(2, minibatch_size, hidden_size).cuda()
          z = 0.1 - 0.2 * torch.rand(minibatch_size, 128).cuda()

          # Generator forward
          g_output = G(g_in, hidden, z)

          # Discriminator forward
          # # Real
          d_real_input = torch.cat([g_in, real], dim=0)               ######CHECHK THE DIM
          # print(d_real_input.shape)
          d_real_prob = D(d_real_input)

          # # Fake
          d_fake_input = torch.cat([g_in, g_output], dim=0)           ######CHECHK THE DIM
          d_fake_prob = D(d_fake_input)

          # Critic forward
          # # Real
          d_real = C(d_real_input)

          # # Fake
          d_fake = C(d_fake_input)

          # Losses
          # ==============================================================
          # ==============================================================
          # Generator losses
          # ==============================================================
          # # Lg = Ladv + alpha*Lconsistency + beta*Lbone
          # # Ladv = -d_fake
          # # Lconsistency = sqrt(sum((yt - yt-1)**2))    l2 loss between current frame and prev frame
          # # Lbone = sum_over_t(square(sum_over_i((bonet - bonegt)**2)))
          # # l2 loss = real future - generated future l2 loss
          # ==============================================================

          g_prev = g_in[input_sequence_length-1:input_sequence_length, :, :]                #####CHECK THIS
          # print(g_prev.shape)
          # print(g_in.shape)
          if output_sequence_length > 1:
              g_prev = torch.cat([g_prev, g_output[0:output_sequence_length-1, :, :]], axis=0)
          g_next = g_output

          # print(g_prev.shape)
          # print(g_next.shape)

          # Consistency loss
                                                      #0.0001  
          g_consistency_loss = torch.max(torch.tensor(0.0001).cuda(), torch.norm(torch.reshape(g_next-g_prev, [-1])) / (minibatch_size*output_sequence_length))
          
          # Ground truth and generator loss
          g_l2_loss = torch.norm(torch.reshape(real - g_output, [-1])) / (minibatch_size*output_sequence_length)
          
          # GAN loss
          g_gan_loss = -torch.mean(d_fake)

          # Total Generator loss
                                #0.001 *
          g_loss = 0.1 * g_gan_loss + 0.5 * g_consistency_loss + 10*g_l2_loss 
                    
          # Update weights
          g_loss.backward(retain_graph = True)

          # Stop
          G_optimizer.step()

          # ===============================================
          # ===============================================
          # Critic Loss
          # ===============================================
          # Lc = Lwgan + lambda*Lgp + alpha*L2
          # Lwgan = d_fake - d_real
          # Lgp = gradient penalty loss
          # L2 = l2 loss
          # ===============================================

          for _ in range(10):
              C_optimizer.zero_grad()
              
              # WGAN loss
              d_wgan_loss = torch.mean(d_fake - d_real)
              
              # d_loss =  + gradient_penalty_loss
              # print("Mean loss", torch.mean(d_fake - d_real).item())
              # print("GP loss", gradient_penalty_loss.item())

              # Gradient Penalty loss
              d_gradient_penalty_loss = 10.0 * gradient_penalty(d_real_input, d_fake_input, C)   

              # Adding L2 loss
              d_l2_loss = 0.001 * torch.sum(torch.tensor([torch.sum(p**2)/2 for p in C.weights()]))
              # d_loss +=        #.weights check
              # writer.add_scalar("discriminator_or_critic_loss", d_loss)
              # print("Weights loss", 0.001 * torch.sum(torch.tensor([p.norm() for p in C.weights()])).item())
              
              # Total loss
              d_loss = d_wgan_loss + d_gradient_penalty_loss + d_l2_loss

              # Update weights
              d_loss.backward(retain_graph = True)

              # Stop
              C_optimizer.step()

              # Critic forward
              # # Real
              d_real = C(d_real_input)

              # # Fake
              d_fake = C(d_fake_input)
          
          # ===============================================
          # ===============================================
          # Discriminator loss
          # ===============================================
          # Ld = Lgan + alpha*L2
          # Lgan = log(d_real) + log(1-d_fake)
          # L2 = L2 loss
          # ===============================================

          # Standard gan loss
          d_gan_loss_prob = -torch.mean(torch.log(d_real_prob) + torch.log(1. - d_fake_prob))
          # d_loss_prob = -torch.mean(torch.log(d_real_prob) + torch.log(1. - d_fake_prob))
          # print("GAN Loss D", d_loss_prob.item())
          # adding L2 loss
          d_l2_loss_prob = 0.001 * torch.sum(torch.tensor([torch.sum(p**2)/2 for p in D.weights()]))
          # print("L2 loss D", 0.001 * torch.sum(torch.tensor([torch.sum(p**2)/2 for p in D.weights()])).item())
          # writer.add_scalar("discriminator_loss", d_loss_prob)

          # Total loss
          d_loss_prob = d_gan_loss_prob + d_l2_loss_prob

          # Update weights
          d_loss_prob.backward()

          # Stop
          D_optimizer.step()
          
          # ===============================================
          # ===============================================
          
          G_losses.append(g_loss.item())
          hist_G_losses.append(g_loss.item())
          
          C_losses.append(d_loss.item())
          hist_C_losses.append(d_loss.item())
          
          D_losses.append(d_loss_prob.item())
          hist_D_losses.append(d_loss_prob.item())

          hist_gcons_losses.append(g_consistency_loss.item())
          hist_gl2_losses.append(g_l2_loss.item())
          hist_ggan_losses.append(g_gan_loss.item())

          # print(len(hist_ggan_losses))

          hist_cw_losses.append(d_wgan_loss.item())
          hist_cgp_losses.append(d_gradient_penalty_loss.item())
          hist_cl2_losses.append(d_l2_loss.item())

          # print(len(hist_cw_losses))

          hist_dgan_losses.append(d_gan_loss_prob.item())
          hist_dl2_losses.append(d_l2_loss_prob.item())

          num_iter += 1

      # Save model
      torch.save({
                 'C': C.state_dict(),
                 'D': D.state_dict(),
                 'G': G.state_dict(),
                 'C_optimizer': C_optimizer.state_dict(),
                 'D_optimizer': D_optimizer.state_dict(),
                 'G_optimizer': G_optimizer.state_dict()
                  }, '/content/drive/Shared drives/Computer_Vision_Final_Project/saved_weights/c05g01l10_epoch20.pt')
      epoch_end_time = time.time()
      per_epoch_ptime = epoch_end_time - epoch_start_time

      print('[%d/%d] - using time: %.2f' % ((epoch + 1), num_epochs, per_epoch_ptime))
      print('loss of critic C: %.3f' % (torch.mean(torch.FloatTensor(C_losses))))
      print('loss of discriminator D: %.3f' % (torch.mean(torch.FloatTensor(D_losses))))
      print('loss of generator G: %.3f' % (torch.mean(torch.FloatTensor(G_losses))))
      
  end_time = time.time()
  total_ptime = end_time - start_time

  return hist_D_losses, hist_G_losses, hist_C_losses, hist_gcons_losses, hist_gl2_losses, hist_ggan_losses,\
  hist_cw_losses, hist_cgp_losses, hist_cl2_losses, hist_dgan_losses, hist_dl2_losses



In [0]:
###TRAINING BLOCK#####
batch_size = 20
inp_seq_len = 10
hidden_size = 1024
out_seq_len = 10
input_size = 66
input_depth = 128

epochs = 5

seq_length = inp_seq_len + out_seq_len

gen = GeneratorRNN(batch_size, hidden_size, input_depth, input_size, inp_seq_len, out_seq_len).cuda()
critic = NNDiscriminator(input_size, input_depth, seq_length).cuda()
disc = NNDiscriminator(input_size, input_depth, seq_length, 'prob').cuda()

hist_D_losses, hist_G_losses, hist_C_losses, hist_gcons_losses, hist_gl2_losses, hist_ggan_losses, hist_cw_losses, hist_cgp_losses, hist_cl2_losses, hist_dgan_losses, hist_dl2_losses = train(gen, disc, critic, epochs, resume=True)

Streaming output truncated to the last 5000 lines.
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteration 78
iteration 79
iteration 80
iteration 81
iteration 82
iteration 83
iteration 84
iteration 85
iteration 86
iteration 87
iteration 88
iteration 89
iteration 90
iteration 91
iteration 92
iteration 93
iteration 94
iteration 95
iteration 96
iteration 97


# **Visualize losses**

In [0]:
print(len(hist_gcons_losses))

20000


In [0]:
# visualize the training / validation accuracies
# epochs = 3complement the advantages of both works
iterations = 1000
x = np.arange(epochs)

# train/val accuracies for MiniVGG
plt.figure()
plt.plot(x, hist_C_losses[::iterations])
plt.plot(x, hist_D_losses[::iterations])
plt.plot(x, hist_G_losses[::iterations])
plt.legend(['critic loss', 'disc loss', 'gen loss'])
plt.xticks(x)
plt.xlabel('Epoch')
plt.ylabel('losses')
plt.title('Losses per epoch')
plt.show()

plt.figure()
plt.plot(x, hist_ggan_losses[::iterations])
plt.plot(x, hist_gcons_losses[::iterations])
plt.plot(x, hist_gl2_losses[::iterations])
plt.plot(x, hist_G_losses[::iterations])
plt.legend(['gan loss', 'con loss', 'l2 loss','total loss'])
plt.xticks(x)
plt.xlabel('iterations')
plt.ylabel('losses')
plt.title('Generator Losses per epoch')
plt.show()

plt.figure()
plt.plot(x, hist_cw_losses[::iterations])
plt.plot(x, hist_cgp_losses[::iterations])
plt.plot(x, hist_cl2_losses[::iterations])
plt.plot(x, hist_C_losses[::iterations])
plt.legend(['wgan loss', 'gp loss', 'l2 loss', 'total loss'])
plt.xticks(x)
plt.xlabel('iterations')
plt.ylabel('losses')
plt.title('Critic Losses per epoch')
plt.show()

plt.figure()
plt.plot(x, hist_dgan_losses[::iterations])
plt.plot(x, hist_dl2_losses[::iterations])
plt.plot(x, hist_D_losses[::iterations])
plt.legend(['gan loss', 'l2 loss', 'total loss'])
plt.xticks(x)
plt.xlabel('iterations')
plt.ylabel('losses')
plt.title('Disc Losses per epoch')
plt.show()


# **Test data**

In [0]:
# Test data
idx = 11553
inp_ = test_dataset[idx]['Input']
exp_out = test_dataset[idx]['Output'][:10,:]

g_in = torch.tensor(inp_ / maxs_, dtype=torch.float32).cuda().unsqueeze(1)
exp_out_norm = torch.tensor(exp_out / maxs_, dtype=torch.float32).cuda().unsqueeze(1)

hidden_size = 1024
hidden = torch.zeros(2, 1, hidden_size, dtype=torch.float32).cuda()
z = 1 - 2 * torch.rand(1, 128).cuda()

checkpoint = torch.load('/content/drive/Shared drives/Computer_Vision_Final_Project/saved_weights/c05g01l10_epoch20.pt')

batch_size = 20
inp_seq_len = 10
hidden_size = 1024
out_seq_len = 10
input_size = 66
input_depth = 128

epochs = 5

seq_length = inp_seq_len + out_seq_len


gen = GeneratorRNN(batch_size, hidden_size, input_depth, input_size, inp_seq_len, out_seq_len).cuda()
gen.load_state_dict(checkpoint['G'])

gen.eval()
pred_out = (gen.forward(g_in, hidden, z)).data

d_inp_r = torch.cat([g_in, exp_out_norm], axis=0)
d_inp_f = torch.cat([g_in, pred_out.unsqueeze(1)], axis=0)

disc = NNDiscriminator(input_size, input_depth, seq_length, 'prob').cuda()
disc.load_state_dict(checkpoint['D'])
disc.eval()

d_pred_prob_r = disc.forward(d_inp_r)
d_pred_prob_f = disc.forward(d_inp_f)

pred_out_np = np.array((pred_out.cpu() * maxs_).data)

print(inp_[0])
print(pred_out_np[0])
print(exp_out[0])

[ 1.2685661  -1.1075445  -1.2441524   0.09377258  0.11016568  0.05273999
 -0.3774081  -0.01644808  0.05747706  0.21635653 -0.05234357 -0.06070291
  0.18024415 -0.08009749 -0.04984165  0.21866968 -0.15279163  0.01230904
 -0.0206693  -0.03105249  0.0375011  -0.41161886  0.19233656  0.08231512
 -0.18083565 -0.19337767  0.04427444  0.02535769 -0.06435853  0.03195276
  0.          0.          0.          0.          0.          0.
 -0.05680607  0.04478429  0.08493309  0.06460758  0.0225683  -0.38954523
  0.00190156 -0.02791121  0.40482855  0.01182799 -0.02280088  0.02052509
  0.08607867 -0.18403743 -0.8507685   0.18524459  0.12425382  0.8921242
  0.40954077 -0.53724027  0.19030403  0.31794527  0.43459377 -0.17441885
  0.15894724 -0.16218777 -0.10403714  0.12849669  0.13892241  0.19256529]
[ 1.26185152 -1.36075545 -1.46420041 -0.31447409 -0.04051051  0.16197834
 -0.03116897 -0.01600785  0.26434919  0.16061384 -0.00725495 -0.22449256
  0.36434124  0.10474779 -0.09557685  0.57818049  0.0249015

In [0]:
np.savez("/content/drive/Shared drives/Computer_Vision_Final_Project/new_pred/bhushan_pred/added_l2_loss/c05g01l10/pred_seq_test_ds_s11553_efinal", pred_out_np)
np.savez("/content/drive/Shared drives/Computer_Vision_Final_Project/new_pred/bhushan_pred/added_l2_loss/c05g01l10/inp_seq_test_ds_s11553_efinal", inp_)
np.savez("/content/drive/Shared drives/Computer_Vision_Final_Project/new_pred/bhushan_pred/added_l2_loss/c05g01l10/exp_seq_test_ds_s11553_efinal", exp_out)


In [0]:
with open('/content/drive/Shared drives/Computer_Vision_Final_Project/new_pred/bhushan_pred/added_l2_loss/c05g01l10/losses_test_ds_s11553.pkl', 'wb') as f:
  pickle.dump({'C':hist_C_losses, 'D':hist_D_losses, 'G':hist_G_losses}, f)